In [1]:
import mne
from mne.io import concatenate_raws, read_raw_edf
import matplotlib.pyplot as plt
import mne.viz
import os
import os.path as op
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs,
                               corrmap)
from mne.time_frequency import tfr_multitaper
from mne.stats import permutation_cluster_1samp_test as pcluster_test
from mne.datasets import fetch_fsaverage
from mne import Epochs, EvokedArray, create_info, io, pick_types, read_events
from mne.datasets import sample
from mne.decoding import Vectorizer
from mne.decoding import CSP

import torch
from torcheeg.models import EEGNet
from torcheeg.models import DGCNN
from torcheeg import transforms

import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import scipy as sc
from matplotlib import cm
import matplotlib.colors as colors

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
import torchaudio

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 


/home/aurelien.stumpf/Development/BCI_Classification/eeg_env/lib/python3.8/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [ ]:
import sys 
sys.path.append("/home/aurelien.stumpf/Development/BCI_Classification/")
from eeg_project_package import dataset, models, spectral_analysis, training

In [ ]:
import importlib
importlib.reload(dataset)
importlib.reload(training)
importlib.reload(models)
importlib.reload(spectral_analysis)

## Check Loading of Braccio Dataset Files

In [ ]:
#Define the parameters
subject = 1  # use data from subject 1
runs = [6, 10, 14]  # use only hand and feet motor imagery runs

#Get data and locate in to given path
input_fname1 =  "/home/aurelien.stumpf/Development/Datasets/physionet.org/files/eegmmidb/1.0.0/S001/S001R04.edf"
#Read raw data files where each file contains a run
raws = read_raw_edf(input_fname1, preload=True)
#Combine all loaded runs
#raw_obj = concatenate_raws(raws)
raw_obj = raws

raw_data = raw_obj.get_data()
events = mne.events_from_annotations(raw_obj,event_id='auto')

print("Number of channels: ", str(len(raw_data)))
print("Number of samples: ", str(len(raw_data)))

#Plot epochs & PSD
raw_obj.plot(duration=120, n_channels=15, scalings=dict(eeg=420e-6))
raw_obj.plot_psd(average=True)

# list of all channel names
list_all_ch_names = raw_obj.ch_names

In [ ]:
# get sampling frequency
sfreq = raw_obj.info['sfreq']
print('Sampling frequency:', sfreq)

In [ ]:
dict_channels = {'Fc5.' : 'FC5',
 'Fc3.' : 'FC3',
 'Fc1.' : 'FC1',
 'Fcz.' : 'FCz',
 'Fc2.' : 'FC2',
 'Fc4.' : 'FC4',
 'Fc6.' : 'FC6',
 'C5..' : 'C5',
 'C3..' : 'C3',
 'C1..' : 'C1',
 'Cz..' : 'Cz',
 'C2..' : 'C2',
 'C4..' : 'C4',
 'C6..' : 'C6',
 'Cp5.' : 'CP5',
 'Cp3.' : 'CP3',
 'Cp1.' : 'CP1',
 'Cpz.' : 'CPz',
 'Cp2.' : 'CP2',
 'Cp4.' : 'CP4',
 'Cp6.' : 'CP6',
 'Fp1.' : 'Fp1',
 'Fpz.' : 'Fpz',
 'Fp2.' : 'Fp2',
 'Af7.' : 'AF7',
 'Af3.' : 'AF3',
 'Afz.' : 'AFz',
 'Af4.' : 'AF4',
 'Af8.' : 'AF8',
 'F7..' : 'F7',
 'F5..' : 'F5',
 'F3..' : 'F3',
 'F1..' : 'F1',
 'Fz..' : 'Fz',
 'F2..' : 'F2',
 'F4..' : 'F4',
 'F6..' : 'F6',
 'F8..' : 'F8',
 'Ft7.' : 'FT7',
 'Ft8.' : 'FT8',
 'T7..' : 'T7',
 'T8..' : 'T8',
 'T9..' : 'T9',
 'T10.' : 'T10',
 'Tp7.' : 'TP7',
 'Tp8.' : 'TP8',
 'P7..' : 'P7',
 'P5..' : 'P5',
 'P3..' : 'P3',
 'P1..' : 'P1',
 'Pz..' : 'Pz',
 'P2..' : 'P2',
 'P4..' : 'P4',
 'P6..' : 'P6',
 'P8..' : 'P8',
 'Po7.' : 'PO7',
 'Po3.' : 'PO3',
 'Poz.' : 'POz',
 'Po4.' : 'PO4',
 'Po8.' : 'PO8',
 'O1..' : 'O1',
 'Oz..' : 'Oz',
 'O2..' : 'O2',
 'Iz..' : 'Iz'}

dict_channels_inv = dict_channels.copy()

# Reverse dict
dict_channels = {v: k for k, v in dict_channels.items()}

## Multi-Subject Classif

In [ ]:
import importlib
importlib.reload(dataset)
importlib.reload(training)
importlib.reload(models)
importlib.reload(spectral_analysis)

# Define the channels
list_name_channels = ["CP1","CP3","CP5","C1","C3","C5","C2","CP2","Cz","FCz","C4","CP4"]
list_idx_channels = [list_all_ch_names.index(dict_channels[ch]) for ch in list_name_channels]

# Define the labels
labels = ['T0', 'T2']

parent_folder_path = "/home/aurelien.stumpf/Development/Datasets/physionet.org/files/eegmmidb/1.0.0/"

num_runs = [4, 8, 12]

dict_crossval = {"test_balanced_accuracy":[], "test_loss":[], "val_balanced_accuracy":[], "val_loss":[], "train_balanced_accuracy":[], "train_loss":[]}

K = 8
num_subjects = list(range(0,109))
for k in range(K-1):
    print(k)

    num_test_subjects = num_subjects[k*int(len(num_subjects)/K):(k+1)*int(len(num_subjects)/K)]
    num_val_subjects = num_subjects[(k+1)*int(len(num_subjects)/K):(k+2)*int(len(num_subjects)/K)]
    num_train_subjects = [x for x in num_subjects if x not in num_test_subjects and x not in num_val_subjects]
    num_sessions = []
    list_labels = ["T0","T2"]

    # Define the parameters of the dataset
    print("Creating the dataset")

    feature_type = ["time"]
    dict_preprocessing = {"polynomial_degree":None,"tmin":0,"tmax":4,"scale":"standard","seq_length":641,"fs":160}
    dict_features = {"type_psd":"welch","fs":500,"nfft":300,"noverlap":150,"nperseg":300,"filter_order":19,"fmin":4,"fmax":30}

    print("training set")
    torch_trainset = dataset.Physio_Dataset_Multi_Subject(parent_folder_path, num_train_subjects, num_runs, list_idx_channels, list_labels, feature_type, dict_preprocessing, dict_features)
    print("val set")
    torch_valset = dataset.Physio_Dataset_Multi_Subject(parent_folder_path, num_val_subjects, num_runs, list_idx_channels, list_labels, feature_type, dict_preprocessing, dict_features)
    print("test set")
    torch_testset = dataset.Physio_Dataset_Multi_Subject(parent_folder_path, num_test_subjects, num_runs, list_idx_channels, list_labels, feature_type, dict_preprocessing, dict_features)

    torch_trainset.transform_dataset_numpy_to_torch()
    torch_trainset.features["time"] = torch_trainset.features["time"].unsqueeze(1)
    torch_valset.transform_dataset_numpy_to_torch()
    torch_valset.features["time"] = torch_valset.features["time"].unsqueeze(1)
    torch_testset.transform_dataset_numpy_to_torch()
    torch_testset.features["time"] = torch_testset.features["time"].unsqueeze(1)

    trainloader = torch.utils.data.DataLoader(torch_trainset, batch_size=16, shuffle=True, num_workers=2)
    valloader = torch.utils.data.DataLoader(torch_valset, batch_size=16, shuffle=True, num_workers=2)
    testloader = torch.utils.data.DataLoader(torch_testset, batch_size=16, shuffle=True, num_workers=2)

    print("Start the training")

    feature_type = "time"
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    criterion = torch.nn.CrossEntropyLoss()
    model = EEGNet(chunk_size=641,
                num_electrodes=12,
                dropout=0.25,
                kernel_1=50,
                kernel_2=25,
                F1=10,
                F2=16,
                D=2,
                num_classes=2)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    #scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 50], gamma=0.1)
    scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lambda epoch: 0.97)
    scheduler_dict = {"MultiplicativeLR" : scheduler}
    logs = training.train_model(model, trainloader, testloader, device, criterion, feature_type, 51, optimizer, scheduler_dict, print_epoch = 5)
    
    # change weights of the model to the best weights
    model.load_state_dict(logs["best_model_weights"])
    model.eval()
    train_balanced_accuracy, train_loss = training.evaluate_classification_model(model,trainloader,device,criterion,feature_type,dataset="Train")
    val_balanced_accuracy, val_loss = training.evaluate_classification_model(model,valloader,device,criterion,feature_type,dataset="Val")
    test_balanced_accuracy, test_loss = training.evaluate_classification_model(model,testloader,device,criterion,feature_type,dataset="Test")
    dict_crossval["test_balanced_accuracy"].append(test_balanced_accuracy)
    dict_crossval["test_loss"].append(test_loss)
    dict_crossval["val_balanced_accuracy"].append(val_balanced_accuracy)
    dict_crossval["val_loss"].append(val_loss)
    dict_crossval["train_balanced_accuracy"].append(train_balanced_accuracy)
    dict_crossval["train_loss"].append(train_loss)

     

In [ ]:
dict_crossval["test_balanced_accuracy"]

In [ ]:
import pickle
with open('../dicts_results/inter_subject_classification_physionet/dict_crossval_12_subjects_8_folds.pkl', 'wb') as f:
    pickle.dump(dict_crossval, f)

# ATCNet

In [ ]:
import importlib
importlib.reload(dataset)
importlib.reload(training)
importlib.reload(models)

In [ ]:
import importlib
importlib.reload(dataset)
importlib.reload(training)
importlib.reload(models)
importlib.reload(spectral_analysis)

# Define the channels
list_name_channels = ["CP1","CP3","CP5","C1","C3","C5","C2","CP2","Cz","FCz","C4","CP4"]
list_idx_channels = [list_all_ch_names.index(dict_channels[ch]) for ch in list_name_channels]

# Define the labels
labels = ['T0', 'T2']

parent_folder_path = "/home/aurelien.stumpf/Development/Datasets/physionet.org/files/eegmmidb/1.0.0/"

num_runs = [4, 8, 12]

#dict_crossval = {"test_balanced_accuracy":[], "test_loss":[], "val_balanced_accuracy":[], "val_loss":[], "train_balanced_accuracy":[], "train_loss":[]}

K = 8
num_subjects = list(range(0,109))
for k in range(2,K-1):
    print(k)

    num_test_subjects = num_subjects[k*int(len(num_subjects)/K):(k+1)*int(len(num_subjects)/K)]
    num_val_subjects = num_subjects[(k+1)*int(len(num_subjects)/K):(k+2)*int(len(num_subjects)/K)]
    num_train_subjects = [x for x in num_subjects if x not in num_test_subjects and x not in num_val_subjects]
    num_sessions = []
    list_labels = ["T0","T2"]

    # Define the parameters of the dataset
    print("Creating the dataset")

    feature_type = ["time"]
    dict_preprocessing = {"polynomial_degree":None,"tmin":0,"tmax":4,"scale":"standard","seq_length":641,"fs":160}
    dict_features = {"type_psd":"welch","fs":500,"nfft":300,"noverlap":150,"nperseg":300,"filter_order":19,"fmin":4,"fmax":30}

    print("training set")
    torch_trainset = dataset.Physio_Dataset_Multi_Subject(parent_folder_path, num_train_subjects, num_runs, list_idx_channels, list_labels, feature_type, dict_preprocessing, dict_features)
    print("val set")
    torch_valset = dataset.Physio_Dataset_Multi_Subject(parent_folder_path, num_val_subjects, num_runs, list_idx_channels, list_labels, feature_type, dict_preprocessing, dict_features)
    print("test set")
    torch_testset = dataset.Physio_Dataset_Multi_Subject(parent_folder_path, num_test_subjects, num_runs, list_idx_channels, list_labels, feature_type, dict_preprocessing, dict_features)

    torch_trainset.transform_dataset_numpy_to_torch()
    torch_trainset.features["time"] = torch_trainset.features["time"].unsqueeze(1)
    torch_valset.transform_dataset_numpy_to_torch()
    torch_valset.features["time"] = torch_valset.features["time"].unsqueeze(1)
    torch_testset.transform_dataset_numpy_to_torch()
    torch_testset.features["time"] = torch_testset.features["time"].unsqueeze(1)

    trainloader = torch.utils.data.DataLoader(torch_trainset, batch_size=16, shuffle=True, num_workers=2)
    valloader = torch.utils.data.DataLoader(torch_valset, batch_size=16, shuffle=True, num_workers=2)
    testloader = torch.utils.data.DataLoader(torch_testset, batch_size=16, shuffle=True, num_workers=2)

    print("Start the training")

    feature_type = "time"
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    criterion = torch.nn.CrossEntropyLoss()
    model = models.ATCNet(in_channels=1,
                num_classes=2,
                num_windows=5,
                num_electrodes=12,
                chunk_size=641,
                filter_size=50,)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler1 = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lambda epoch: 0.95)
    scheduler2 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
    scheduler_dict = {"MultiplicativeLR":scheduler1} #, "ReduceLROnPlateau":scheduler2}
    logs = training.train_model(model, trainloader, testloader, device, criterion, feature_type, 51, optimizer, scheduler_dict, print_epoch = 5)
    
    # change weights of the model to the best weights
    model.load_state_dict(logs["best_model_weights"])
    model.eval()
    train_balanced_accuracy, train_loss = training.evaluate_classification_model(model,trainloader,device,criterion,feature_type,dataset="Train")
    val_balanced_accuracy, val_loss = training.evaluate_classification_model(model,valloader,device,criterion,feature_type,dataset="Val")
    test_balanced_accuracy, test_loss = training.evaluate_classification_model(model,testloader,device,criterion,feature_type,dataset="Test")
    dict_crossval["test_balanced_accuracy"].append(test_balanced_accuracy)
    dict_crossval["test_loss"].append(test_loss)
    dict_crossval["val_balanced_accuracy"].append(val_balanced_accuracy)
    dict_crossval["val_loss"].append(val_loss)
    dict_crossval["train_balanced_accuracy"].append(train_balanced_accuracy)
    dict_crossval["train_loss"].append(train_loss)

In [ ]:
feature_type = "time"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = torch.nn.CrossEntropyLoss()
model = models.ATCNet(in_channels=1,
               num_classes=2,
               num_windows=5,
               num_electrodes=12,
               chunk_size=641,
               filter_size=50,)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler1 = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lambda epoch: 0.95)
scheduler2 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
scheduler_dict = {"MultiplicativeLR":scheduler1} #, "ReduceLROnPlateau":scheduler2}
logs = training.train_model(model, trainloader, testloader, device, criterion, feature_type, 101, optimizer, scheduler_dict, print_epoch = 1)

In [ ]:
li_eegnet = np.array([0.75, 0.75, 0.77])
li_atcnet = np.array([0.77, 0.76, 0.78])
print(np.mean(li_eegnet), np.mean(li_atcnet))
print(np.std(li_eegnet), np.std(li_atcnet))

In [ ]:
model

In [ ]:
for param in model.parameters():
    print(param.data.shape)
    time_filters = param.data.cpu().numpy()
    break

for i in range(16):
    plt.plot(time_filters[i,0,0,:])
    plt.show()

In [ ]:
model

In [ ]:
fig, ax = plt.subplots(4, 4, figsize=(10, 10))

for i in range(4):
    for j in range(4):
        spatial_map_12 = model.conv_block[2].weight.data[i + 4 * j].cpu().numpy().flatten()
        spatial_map_64 = np.zeros(64)
        spatial_map_64[list_idx_channels] = spatial_map_12
        mne.viz.plot_topomap(
            spatial_map_64,
            ch_positions[:, :2],
            show=False,
            axes=ax[i, j]
        )

plt.show()

# LDA / SVM

In [ ]:
X_train = torch_trainset.features["band_psd"].reshape(torch_trainset.features["band_psd"].shape[0], -1)
y_train = torch_trainset.labels
X_test = torch_testset.features["band_psd"].reshape(torch_testset.features["band_psd"].shape[0], -1)
y_test = torch_testset.labels

# Train LDA classifier
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(X_train, y_train)
y_pred = lda_clf.predict(X_test)
acc = np.mean(y_pred == y_test)
print("LDA accuracy: ", acc)

# Train SVM classifier
svm_clf = SVC(kernel='linear')
svm_clf.fit(X_train, y_train)
y_pred = svm_clf.predict(X_test)
acc = np.mean(y_pred == y_test)
print("SVM accuracy: ", acc)

# Train SVM classifier
svm_clf = SVC(kernel='rbf')
svm_clf.fit(X_train, y_train)
y_pred = svm_clf.predict(X_test)
acc = np.mean(y_pred == y_test)
print("rbf SVM accuracy: ", acc)

# Deep Fourier Transform

In [ ]:
torch_trainset.features["time"].shape

In [ ]:
import importlib
importlib.reload(dataset)
importlib.reload(training)
importlib.reload(models)

In [ ]:
torch_trainset.transform_dataset_numpy_to_torch()
torch_trainset.features["time"] = torch_trainset.features["time"][:,0,:].squeeze(1)
torch_testset.transform_dataset_numpy_to_torch()
torch_testset.features["time"] = torch_testset.features["time"][:,0,:].squeeze(1)

In [ ]:
feature_type = "time"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = torch.nn.BCEWithLogitsLoss() #CrossEntropyLoss()
model = models.DeepWelchTransform(nperseg = 400, noverlap = 100, seq_length = 641)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler1 = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lambda epoch: 0.95)
scheduler2 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
scheduler_dict = {"MultiplicativeLR":scheduler1} #, "ReduceLROnPlateau":scheduler2}
logs = training.train_model(model, trainloader, testloader, device, criterion, feature_type, 101, optimizer, scheduler_dict, print_epoch = 10)

In [ ]:
criterion.__class__.__name__